# Assignment 5: Anomaly Detection

## Objective

In this assignment, you will be instructed to apply **unsupervised** learning to address a practical problem. For simplicity, we will use intrusion detection as an example. But the principle can be used in many other fields, such as fraud detection and medical care. After completing this assignment, you should be able to answer the following questions:

1. How to standardize numerical features?
2. How to transform categorical features into numerical features?
3. How to derive anomalies from clustering results?
4. How to tune parameters for unsupervised learning?

The data can be downloaded from [A5-data.zip](A5-data.zip).

### Overview

In practice, when you are facing an ML problem, the process of solving it basically consists of two phases: _model development_ and _model serving_. 

* In model development, your job is to figure out what's the best algorithms, features, and parameters should be chosen based on historical data. This is often an iterative and off-line process. 

* Once you develop a satisfactory model, you will need to use the model to serve new requests and make predictions. This is often an on-line process, so you have to think about how to make the predictions as fast as possible and how to efficiently update the model when new data arrive. 

In this assignment, you are going to focus on the model development stage. Model serving is also a very important topic. I hope you can learn it by yourself or through your final project. Below are a few good references: 
* [Deploy a Model in Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html)
* [TensorFlow Serving](https://www.tensorflow.org/tfx/guide/serving)
* [Deploy models with Azure Machine Learning](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where)
* [Serving of ML models in Kubeflow](https://www.kubeflow.org/docs/components/serving/tfserving_new/)
* [Announcing MLflow Model Serving on Databricks](https://databricks.com/blog/2020/06/25/announcing-mlflow-model-serving-on-databricks.html)

### Model Development

Suppose you want to develop a model that can detect anomalous connections to your company's server. The server log contains all the information of historical connections; your nice colleague has already helped you to transform the raw log into a collection of feature vectors, where each feature vector characterizes a connection in 40 dimensions, e.g., number of failed login attempts, length (number of seconds) of the connection. Here is one example feature vector:
```
[udp,SF,0,105,146,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0.00,0.00,0.00,0.00,1.00,0.00,0.00,255,240,0.94,0.01,0.00,0.00,0.00,0.00,0.00,0.00]
```

Your task is to take these feature vectors as input and develop an unsupervised-learning model to detect anomalous connections. In the lecture, we have gone through this process. In the assignment, you are going to implement three functions: `cat2Num`, `scaleNum`, and `detect`, by doing Tasks A-C, respectively.

``` python
# anomaly_detection.py
import pandas as pd


class AnomalyDetection():
    
    def scaleNum(self, df, indices):
        """
            Write your code!
        """
    
    
    def cat2Num(self, df, indices):
        """
            Write your code!
        """
            

    def detect(self, df, k, t):
        """
            Write your code!
        """
        
    
 
if __name__ == "__main__":
    df = pd.read_csv('logs-features-sample.csv').set_index('id')
    ad = AnomalyDetection()
    
    df1 = ad.cat2Num(df, [0,1])
    print(df1)

    df2 = ad.scaleNum(df1, [6])
    print(df2)

    df3 = ad.detect(df2, 8, 0.97)
    print(df3)
```

### A toy dataset for testing

To test your program, you can read a toy dataset:

```python
    data = [(0, ["http", "udt", 4]), \
            (1, ["http", "udf", 5]), \
            (2, ["http", "tcp", 5]), \
            (3, ["ftp", "icmp", 1]), \
            (4, ["http", "tcp", 4])]

    df = pd.DataFrame(data=data, columns = ["id", "features"])
        
```

After running the script, your program should output the followings:

**df1 = ad.cat2Num(df, [0,1])**

```
+---+---------------------+
|id |features             |
+---+---------------------+
|0  |[1, 0, 1, 0, 0, 0, 4]|
|1  |[1, 0, 0, 1, 0, 0, 5]|
|2  |[1, 0, 0, 0, 1, 0, 5]|
|3  |[0, 1, 0, 0, 0, 1, 1]|
|4  |[1, 0, 0, 0, 1, 0, 4]|
+---+---------------------+
```

**df2 = ad.scaleNum(df1, [6]):**

```
+---+---------------------------------------+
|id |features                               |
+---+---------------------------------------+
|0  |[1, 0, 1, 0, 0, 0, 0.12171612389003701]|
|1  |[1, 0, 0, 1, 0, 0, 0.7302967433402214] |
|2  |[1, 0, 0, 0, 1, 0, 0.7302967433402214] |
|3  |[0, 1, 0, 0, 0, 1, -1.704025734460517] |
|4  |[1, 0, 0, 0, 1, 0, 0.12171612389003701]|
+---+---------------------------------------+
```

**df3 = ad.detect(df2, 2, 0.9)**

```
+---+---------------------------------------+-----+
|id |features                               |score|
+---+---------------------------------------+-----+
|3  |[0, 1, 0, 0, 0, 1, -1.704025734460517] |1.0  |
+---+----------------+----------------------+-----+
```

### Task A. Categorical Features --> Numerical Features

As you can see from the above feature vector, the first two dimensions in each feature vector are categorical features. For example, the first dimension can be one of the following cases: “http” or “ftp”. You can represent these categorical features using one-hot encoding, e.g., [1,0] for “http” and [0,1] for “ftp”. 

In Task A, your job is to implement the `cat2Num` function.

```python
    def cat2Num(self, df, indices):
        """ 
            Input: $df represents a DataFrame with two columns: "id" and "features"
                   $indices represents which dimensions in $features are categorical features, 
                    e.g., indices = [0, 1] denotes that the first two dimensions are categorical features.
                    
            Output: Return a new DataFrame that updates the "features" column with one-hot encoding. 
            
        """
    
```

In [452]:
from ast import literal_eval
df_test = pd.read_csv('A5-data/logs-features-sample.csv').set_index('id')
df_test['features'] = df_test['features'].apply(literal_eval)

In [466]:
df_test.features.str[0]

id
0        tcp
1        tcp
2        tcp
3        tcp
4        udp
        ... 
99090    tcp
99091    udp
99092    tcp
99093    tcp
99094    tcp
Name: features, Length: 99095, dtype: object

In [467]:
df_test.head()

,features
id,
0,"[tcp, SF, -0.157817888091, 0.015677505748, -0...."
1,"[tcp, SF, -0.158545578037, -0.0282581926784, -..."
2,"[tcp, SF, -0.158545578037, -0.0261474606772, 0..."
3,"[tcp, SF, -0.158545578037, 0.0132420457465, -0..."
4,"[udp, SF, -0.158545578037, -0.0314405270803, -..."


In [266]:
df1 = cat2Num(df_test, [0,1])

In [272]:
df1.features

id
0        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.157817...
1        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
2        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
3        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
4        [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
                               ...                        
99090    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
99091    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1.7552789...
99092    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
99093    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -0.158545...
99094    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
Name: features, Length: 99095, dtype: object

In [800]:
df2 = cat2Num(df_test, [0,1])

In [809]:
df2.features

id
0        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.157817...
1        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
2        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
3        [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
4        [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
                               ...                        
99090    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
99091    [0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1.7552789...
99092    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
99093    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -0.158545...
99094    [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545...
Name: features, Length: 99095, dtype: object

In [810]:
df3 = scaleNum(df2, [12])

In [813]:
df3

,features
id,
0,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.157817..."
1,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."
2,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."
3,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."
4,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."
...,...
99090,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."
99091,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1.7552789..."
99092,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.158545..."


In [797]:
import numpy as np
from collections import defaultdict 
def cat2Num(df, indices):
    field_list = defaultdict(list) 
    df_copy = df.copy()
    df_copy['new_features'] = [[] for _ in range(len(df))]
    for i in indices:
        field_list[i] = df.features.str[i]
    dict = {}
    for i in field_list:
        field_list[i] = list(set(field_list[i]))
    for i in indices:
        unique_list = field_list[i]
        for record in unique_list:
            encode = helper2(record, unique_list)
            dict[record] = encode
    for i in indices:
        current_list = df.features.str[i]
        res = []
        for item in current_list:
            res.append(dict[item])
        df_copy['feature'+str(i)] = pd.Series([x for x in res])
    num_feature = len(df['features'][0])
    for i in range(num_feature):
        if i in indices:
            df_copy['new_features'] = df_copy['new_features'] + df_copy['feature' + str(i)]
        else:
            df_copy['new_features'] = df_copy['new_features'] + pd.Series([[x] for x in df['features'].str[i]])
    del df_copy['features']
    df_copy.rename(columns={"new_features": "features"}, inplace = True)
    for i in indices:
        del df_copy['feature'+str(i)]
    return df_copy

In [803]:
import pandas as pd
data = [(0, ["http", "udt", 4]), \
            (1, ["http", "udf", 5]), \
            (2, ["http", "tcp", 5]), \
            (3, ["ftp", "icmp", 1]), \
            (4, ["http", "tcp", 4])]

df = pd.DataFrame(data=data, columns = ["id", "features"])


In [ ]:
def helper2(record, field):
    char_to_int = dict((c, i) for i, c in enumerate(field))
    int_to_char = dict((i, c) for i, c in enumerate(field))
    integer_encoded = char_to_int[record]
    letter = [0 for _ in range(len(field))]
    letter[integer_encoded] = 1
    return letter

**Note**: You have to implement one-hot encoding by yourself rather than use an existing implmentation in a libary. 


### Task B. Standardize Numerical Features 

K-Means utilizes the distance between every two points to do clustering. We don't want the distance to be dominated by some features. To avoid this, we standardize those features that are on a larger scale. 

In Task B, your job is to implement the `scaleNum` function, which standardizes specified features by removing the mean and scaling to unit variance.


```python
    def scaleNum(self, df, indices):
        """ 
            Input: $df represents a DataFrame with two columns: "id" and "features"
                   $indices represents which dimensions in $features that need to be standardized
                    
            Output: Return a new DataFrame that updates "features" column with specified features standarized.
            
        """
    
```

**Note:** You have to implement StandardScaler by yourself rather than use an existing implmentation in a libary

In [804]:
df1 = cat2Num(df, [0, 1])
df1

,id,features
0,0,"[0, 1, 0, 1, 0, 0, 4]"
1,1,"[0, 1, 0, 0, 0, 1, 5]"
2,2,"[0, 1, 0, 0, 1, 0, 5]"
3,3,"[1, 0, 1, 0, 0, 0, 1]"
4,4,"[0, 1, 0, 0, 1, 0, 4]"


In [807]:
def scaleNum(df, indices):
    df_copy = df.copy()
    df_copy['new_features'] = [[] for _ in range(len(df))]
    for i in indices:
        col = df.features.str[i]
        col_mean = np.mean(df.features.str[i])
        col_std = np.std(df.features.str[i], ddof = 1)
        scaled = (df.features.str[i] - col_mean) / col_std
        df_copy['scale'+str(i)] = pd.Series([[x] for x in scaled])
    num_feature = len(df['features'][0])
    for i in range(num_feature):
        if i in indices:
            df_copy['new_features'] = df_copy['new_features'] + df_copy['scale' + str(i)]
        else:
            df_copy['new_features'] = df_copy['new_features'] + pd.Series([[x] for x in df['features'].str[i]])

    del df_copy['features']
    df_copy.rename(columns={"new_features": "features"}, inplace = True)
    for i in indices:
        del df_copy['scale'+str(i)]
    return df_copy

In [814]:
df_scale = scaleNum(df1, [6])
df_scale

,id,features
0,0,"[0, 1, 0, 1, 0, 0, 0.12171612389003701]"
1,1,"[0, 1, 0, 0, 0, 1, 0.7302967433402215]"
2,2,"[0, 1, 0, 0, 1, 0, 0.7302967433402215]"
3,3,"[1, 0, 1, 0, 0, 0, -1.7040257344605165]"
4,4,"[0, 1, 0, 0, 1, 0, 0.12171612389003701]"


### Task C. Detect Anomalies

As you may remember, the intuition of our anomaly detection approach was that clusters with a small number of data points will correspond to attacks or anomalies. We use this intuition to generate a confidence score from the clustering model’s output. The confidence score reflects how much the clustering model believes a data point is an attack or not. Let us assume $x$ is a data point describing a network connection. We can use:

$$score(x) = \frac{N_{max}-N_{x}}{N_{max}-N_{min}}$$

to score $x$ as being an anomaly. Note that in this equation, $N_{max}$ and $N_{min}$ reflect the size of the largest and smallest clusters, respectively. $N_{x}$ represents the size of the cluster assigned to $x$. If you check the equation carefully, you will notice that $score(x) = 1$ when $x$ is assigned to the smallest cluster and $score(x)$ = 0 when $x$ is assigned to a large cluster.


In Task C, your job is to implement the `detect` function. 

```python
def detect(self, df, k, t):
    """ 
        Input: $df represents a DataFrame with two columns: "id" and "features"
               $k is the number of clusters for K-Means
               $t is the score threshold
        
        Output: Return a new DataFrame that adds the "score" column into the input $df and then
                removes the rows whose scores are smaller than $t.  
    """
```

In [831]:
df_scale

,id,features
0,0,"[0, 1, 0, 1, 0, 0, 0.12171612389003701]"
1,1,"[0, 1, 0, 0, 0, 1, 0.7302967433402215]"
2,2,"[0, 1, 0, 0, 1, 0, 0.7302967433402215]"
3,3,"[1, 0, 1, 0, 0, 0, -1.7040257344605165]"
4,4,"[0, 1, 0, 0, 1, 0, 0.12171612389003701]"


In [846]:
detect(df2, 8, 0.97)

,features,score
id,,
38,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.141808...",0.992259
41,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -0.158545...",0.989859
62,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.139625...",0.992259
367,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.139625...",0.992259
382,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.140353...",0.992259
...,...,...
98198,"[0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, -0.158545...",0.989859
98395,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.143991...",0.992259
98567,"[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, -0.137442...",0.992259


In [839]:
from sklearn.cluster import KMeans
def detect(df, k, t):
    df_copy = df.copy()
    kmeans = KMeans(n_clusters=k, random_state=0).fit(df['features'].to_list())
    labels = kmeans.labels_
    dict = {}
    for i in labels:
        if i in dict:
            dict[i] += 1
        else:
            dict[i] = 1
    n_max = 0
    n_min = len(df['features'])
    for i in range(k):
        if dict[i] > n_max:
            n_max = dict[i]
        if dict[i] < n_min:
            n_min = dict[i]
    scores = []
    for label in labels:
        n = dict[label]
        score = (n_max - n) / (n_max - n_min)
        scores.append(score)
    df_copy['score'] = pd.Series([x for x in scores])
    df_copy = df_copy.loc[(df_copy["score"] > t)]
    return df_copy
        

### Task D. Hyperparameter Tuning

Hyperparameter tuning plays an essential role in improving model quality. In this assignment, your task is to figure out how to tune hyperparameters for **unsupervised learning**. There are two parameters that need to be tuned: 

* $k$: the number of clusters
* $t$: the score threshold 

They are set to k = 8 and t = 0.97 in the above program. In fact, if you changed them to different values, the result could be quite different. Thus, it is important to know how to tune $k$ and $t$ in practice.  

In Task D, imagine you are a data science manager. Please write an email (< 500 words) to tell a junior data scientist (named Nick) how to use [Bayesian Optimization](http://krasserm.github.io/2018/03/21/bayesian-optimization/) to tune the parameters. Your email needs to explain to Nick i) how bayesian optimization works in an intuitive way, and ii) why bayesian optimization could perform better than grid search and random search.


## Submission

Implement `cat2Num`, `scaleNum`, and `detect` functions in `anomaly_detection.py`. Submit your code file (`anomaly_detection.py`), and your email content (`email.pdf`) to CourSys.